### Analysis description
Participants were invited to report the direction of a 1/f noise pattern that changed as a function <br/>
of the parameter Kappa controling the dispersion of the visual orientation contained within the pattern.<br/>
We here compute the perfomance as a function of the exprimental conditions (fullscreen vs. gaze center vs. <br/>
gaze left vs. gaze right; attend-fix, attend-bar) per trials or across trials .<br/>

In [5]:
# Imports
import os
import numpy as np
import pandas as pd
import ipdb
import bids
import warnings
warnings.filterwarnings('ignore')

# Figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.express as px
from plot_utils import plotly_template

# Define parameters
subjects = ['sub-001','sub-002','sub-003','sub-004',
            'sub-005','sub-006','sub-007','sub-008']
subjects_plot = ['sub-001','sub-002','sub-003','sub-004',
                 'sub-005','sub-006','sub-007','sub-008','group']
gaze_tasks = ['FullScreen','GazeCenter','GazeLeft','GazeRight']
attend_tasks = ['AttendFix','AttendBar']

# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
behav_dir = "{}/derivatives/behav_data".format(base_dir)

# Get BIDS files
bids.config.set_option('extension_initial_dot', True)
layout = bids.BIDSLayout(bids_dir, 'synthetic')

### Compute results performance and staircase as a function of trials and gaze/attention conditions
#### Individual level

In [ ]:
# Compute per participant
for subject in subjects:    
    # get filenames
    FullScreen_AttendFix_fn = layout.get(subject=subject[4:], suffix='events', return_type='filename', task='AttendFixGazeCenterFS')
    GazeCenter_AttendFix_fn = layout.get(subject=subject[4:], suffix='events', return_type='filename', task='AttendFixGazeCenter')
    GazeLeft_AttendFix_fn = layout.get(subject=subject[4:] ,suffix='events', return_type='filename', task='AttendFixGazeLeft')
    GazeRight_AttendFix_fn = layout.get(subject=subject[4:], suffix='events', return_type='filename', task='AttendFixGazeRight')
    FullScreen_AttendBar_fn = layout.get(subject=subject[4:], suffix='events', return_type='filename', task='AttendStimGazeCenterFS')
    GazeCenter_AttendBar_fn = layout.get(subject=subject[4:], suffix='events', return_type='filename', task='AttendStimGazeCenter')
    GazeLeft_AttendBar_fn = layout.get(subject=subject[4:], suffix='events', return_type='filename', task='AttendStimGazeLeft')
    GazeRight_AttendBar_fn = layout.get(subject=subject[4:], suffix='events', return_type='filename', task='AttendStimGazeRight')

    tasks_df = pd.DataFrame({'subject': [subject]})
    for gaze_task in gaze_tasks:
        for attend_task in attend_tasks:

            exec("fn = {}_{}_fn".format(gaze_task,attend_task))

            if gaze_task == 'FullScreen': 
                # 4 runs
                run1_df, run2_df = pd.read_csv(fn[0],sep="\t"), pd.read_csv(fn[1],sep="\t") 
                run3_df, run4_df = pd.read_csv(fn[2],sep="\t"), pd.read_csv(fn[3],sep="\t") 

                perf_run1, perf_run2 = np.zeros(run1_df.shape[0]), np.zeros(run2_df.shape[0]) 
                perf_run3, perf_run4 = np.zeros(run3_df.shape[0]), np.zeros(run4_df.shape[0])
                for t in np.arange(run1_df.shape[0]):
                    perf_run1[t],perf_run2[t] = np.nanmean(run1_df.response_val[0:t]), np.nanmean(run2_df.response_val[0:t])
                    perf_run3[t],perf_run4[t] = np.nanmean(run3_df.response_val[0:t]), np.nanmean(run4_df.response_val[0:t])
                perf_runs = [perf_run1,perf_run2,perf_run3,perf_run4]

                if attend_task == 'AttendFix': 
                    stair_run1, stair_run2 = run1_df.fix_stair_val, run2_df.fix_stair_val
                    stair_run3, stair_run4 = run3_df.fix_stair_val, run4_df.fix_stair_val
                elif attend_task == 'AttendBar': 
                    stair_run1, stair_run2 = run1_df.stim_stair_val, run2_df.stim_stair_val
                    stair_run3, stair_run4 = run3_df.stim_stair_val, run4_df.stim_stair_val
                    
                stair_runs = [stair_run1,stair_run2,
                              stair_run3,stair_run4]

                df = pd.DataFrame({ 'trial': run1_df.trial_number,
                                    'perf_run1': perf_run1, 'perf_run2': perf_run2, 
                                    'perf_run3': perf_run3, 'perf_run4': perf_run4,
                                    'perf_mean': np.nanmean(perf_runs,axis=0),
                                    'perf_std': np.nanstd(perf_runs,axis=0),
                                    'stair_run1': stair_run1, 'stair_run2': stair_run2, 
                                    'stair_run3': stair_run3, 'stair_run4': stair_run4,
                                    'stair_mean': np.nanmean(stair_runs,axis=0),
                                    'stair_std': np.nanstd(stair_runs,axis=0)})                
                
            else:
                # 2 runs
                run1_df, run2_df = pd.read_csv(fn[0],sep="\t"), pd.read_csv(fn[1],sep="\t") 
                perf_run1, perf_run2 = np.zeros(run1_df.shape[0]), np.zeros(run2_df.shape[0]) 
                for t in np.arange(run1_df.shape[0]):
                    perf_run1[t] = np.nanmean(run1_df.response_val[0:t])
                    perf_run2[t] = np.nanmean(run2_df.response_val[0:t])
                perf_runs = [perf_run1,perf_run2]

                if attend_task == 'AttendFix': 
                    stair_run1, stair_run2 = run1_df.fix_stair_val, run2_df.fix_stair_val
                elif attend_task == 'AttendBar': 
                    stair_run1, stair_run2 = run1_df.stim_stair_val, run2_df.stim_stair_val
                stair_runs = [stair_run1,stair_run2]

                df = pd.DataFrame({ 'trial': run1_df.trial_number,
                                    'perf_run1': perf_run1, 'perf_run2': perf_run2,
                                    'perf_mean': np.nanmean(perf_runs,axis=0),
                                    'perf_std': np.nanstd(perf_runs,axis=0),
                                    'stair_run1': stair_run1, 'stair_run2': stair_run2, 
                                    'stair_mean': np.nanmean(stair_runs,axis=0),
                                    'stair_std': np.nanstd(stair_runs,axis=0)})

            # save data
            try: os.makedirs("{}/{}".format(behav_dir,subject))
            except: pass
            df_fn = "{}/{}/{}_task-{}{}_behav.tsv".format(behav_dir,subject,subject,gaze_task,attend_task)
            print('saving {}'.format(df_fn))
            df.to_csv(df_fn, sep="\t", na_rep='NaN',index=False)
            
            # compute across trials
            tasks_df['{}{}_stair_mean'.format(gaze_task,attend_task)] = np.nanmean(df.stair_mean)
            tasks_df['{}{}_stair_std'.format(gaze_task,attend_task)] = np.nanstd(df.stair_mean)
            tasks_df['{}{}_perf_mean'.format(gaze_task,attend_task)] = np.nanmean(df.perf_mean)
            tasks_df['{}{}_perf_std'.format(gaze_task,attend_task)] = np.nanstd(df.perf_mean)
            
    
    tasks_df_fn = "{}/{}/{}_behav.tsv".format(behav_dir,subject,subject)
    print('saving {}'.format(tasks_df_fn))
    tasks_df.to_csv(tasks_df_fn, sep="\t", na_rep='NaN',index=False)

#### Group level

In [ ]:
# Compute across participants
for task_num, gaze_task in enumerate(gaze_tasks):
    for subject_num, subject in enumerate(subjects):

        sub_af_fn = "{}/{}/{}_task-{}AttendFix_behav.tsv".format(behav_dir, subject, subject, gaze_task, attend_task)
        sub_ab_fn = "{}/{}/{}_task-{}AttendBar_behav.tsv".format(behav_dir, subject, subject, gaze_task, attend_task)
        sub_af_df = pd.read_csv(sub_af_fn, sep="\t")
        sub_ab_df = pd.read_csv(sub_ab_fn, sep="\t")        
        if subject_num ==0:
            group_af_df = pd.DataFrame({'trial': sub_af_df.trial})
            group_ab_df = pd.DataFrame({'trial': sub_ab_df.trial})
            
            group_af_stair_mat = np.array(sub_af_df.stair_mean).reshape(-1,1)
            group_ab_stair_mat = np.array(sub_ab_df.stair_mean).reshape(-1,1)
            group_af_perf_mat = np.array(sub_af_df.perf_mean).reshape(-1,1)
            group_ab_perf_mat = np.array(sub_ab_df.perf_mean).reshape(-1,1)

        group_af_stair_mat = np.append(group_af_stair_mat, np.array(sub_af_df.stair_mean).reshape(-1,1), axis = 1)
        group_ab_stair_mat = np.append(group_ab_stair_mat, np.array(sub_ab_df.stair_mean).reshape(-1,1), axis = 1)
        group_af_perf_mat = np.append(group_af_perf_mat, np.array(sub_af_df.perf_mean).reshape(-1,1), axis = 1)
        group_ab_perf_mat = np.append(group_ab_perf_mat, np.array(sub_ab_df.perf_mean).reshape(-1,1), axis = 1)
            
    # compute mean across subs
    group_af_df['stair_mean'] = np.nanmean(group_af_stair_mat,axis=1)
    group_af_df['stair_sem'] = np.nanstd(group_af_stair_mat,axis=1)/np.sqrt(len(subjects)-1)
    group_ab_df['stair_mean'] = np.nanmean(group_ab_stair_mat,axis=1)
    group_ab_df['stair_sem'] = np.nanstd(group_ab_stair_mat,axis=1)/np.sqrt(len(subjects)-1)
    group_af_df['perf_mean'] = np.nanmean(group_af_perf_mat,axis=1)
    group_af_df['perf_sem'] = np.nanstd(group_af_perf_mat,axis=1)/np.sqrt(len(subjects)-1)
    group_ab_df['perf_mean'] = np.nanmean(group_ab_perf_mat,axis=1)
    group_ab_df['perf_sem'] = np.nanstd(group_ab_perf_mat,axis=1)/np.sqrt(len(subjects)-1)

    # save data
    try: os.makedirs("{}/group".format(behav_dir))
    except: pass

    group_af_df_fn = "{}/group/group_task-{}AttendFix_behav.tsv".format(behav_dir,gaze_task)
    print('saving {}'.format(group_af_df_fn))
    group_af_df.to_csv(group_af_df_fn, sep="\t", na_rep='NaN',index=False)

    group_ab_df_fn = "{}/group/group_task-{}AttendBar_behav.tsv".format(behav_dir,gaze_task)
    print('saving {}'.format(group_ab_df_fn))
    group_ab_df.to_csv(group_ab_df_fn, sep="\t", na_rep='NaN',index=False)
    
for subject_num, subject in enumerate(subjects):
    sub_fn = "{}/{}/{}_behav.tsv".format(behav_dir, subject, subject)
    sub_df = pd.read_csv(sub_fn, sep="\t")

    if subject_num ==0: mat_df = [sub_df]
    else: mat_df.append(sub_df)
    
    subjects_df = pd.concat(mat_df, sort=False)
    
group_tasks_df = pd.DataFrame({'subject': ['group']})
for gaze_task in gaze_tasks:
    for attend_task in attend_tasks:
        group_tasks_df['{}{}_stair_mean'.format(gaze_task,attend_task)] = subjects_df['{}{}_stair_mean'.format(gaze_task,attend_task)].mean()
        group_tasks_df['{}{}_stair_sem'.format(gaze_task,attend_task)] = subjects_df['{}{}_stair_mean'.format(gaze_task,attend_task)].sem()
        group_tasks_df['{}{}_perf_mean'.format(gaze_task,attend_task)] = subjects_df['{}{}_perf_mean'.format(gaze_task,attend_task)].mean()
        group_tasks_df['{}{}_perf_sem'.format(gaze_task,attend_task)] = subjects_df['{}{}_perf_mean'.format(gaze_task,attend_task)].sem()
        
group_tasks_df_fn = "{}/group/group_behav.tsv".format(behav_dir)
print('saving {}'.format(group_tasks_df_fn))
group_tasks_df.to_csv(group_tasks_df_fn, sep="\t", na_rep='NaN',index=False)

#### Plot performance and staircase across trials

In [6]:
# General figure settings
template_specs = dict(  axes_color="rgba(0, 0, 0, 1)",
                        axes_width=2,
                        axes_font_size=13,
                        bg_col="rgba(255, 255, 255, 1)",
                        font='Arial',
                        title_font_size=15,
                        plot_width=1.5)
fig_template = plotly_template(template_specs)

y_range_stair = [0,15]
y_range_perf = [0,1]
ab_line_col = "rgba(0, 0, 0, 1)"
af_line_col = "rgba(157, 157, 157, 1)"
ab_area_col = "rgba(0, 0, 0, 0.3)"
af_area_col = "rgba(157, 157, 157, 0.3)"
tasks_label = ['Full<br>screen', 'Gaze<br>center', 'Gaze<br>left', 'Gaze<br>right']

# Subplot settings
rows, cols = 2, 5
fig_height, fig_width = 500,1250
column_widths,row_heights = [1,1,1,1,1],[1,1]
sb_specs = [[{},{},{},{},{}],[{},{},{},{},{}]]

hovertemplate_stair = 'Trial: %{x:2.0f}<br>' + 'Kappa: %{y:2.2f}'
hovertemplate_perf = 'Trial: %{x:2.0f}<br>' + 'Perf: %{y:1.2f}'
hovertemplate_stair_task = 'Task: %{x}<br>' + 'Kappa: %{y:2.2f}'
hovertemplate_perf_task = 'Task: %{x}<br>' + 'Perf: %{y:1.2f}'

for subject in subjects_plot:
        
    subplot_titles = ('<b>Full screen</b> ({})'.format(subject), 
                      '<b>Gaze center</b> ({})'.format(subject),
                      '<b>Gaze left</b> ({})'.format(subject), 
                      '<b>Gaze right</b> ({})'.format(subject),
                      '<b>Behavior</b> ({})'.format(subject))

    fig = make_subplots(rows=rows, cols=cols, specs=sb_specs, print_grid=False, vertical_spacing=0.15, horizontal_spacing=0.05, 
                    column_widths=column_widths, row_heights=row_heights,  subplot_titles=subplot_titles)
    for task_num, gaze_task in enumerate(gaze_tasks):
        
        
        af_fn = "{}/{}/{}_task-{}AttendFix_behav.tsv".format(behav_dir, subject, subject, gaze_task, attend_task)
        ab_fn = "{}/{}/{}_task-{}AttendBar_behav.tsv".format(behav_dir, subject, subject, gaze_task, attend_task)
        af_df = pd.read_csv(af_fn, sep="\t")
        ab_df = pd.read_csv(ab_fn, sep="\t")
        
        tasks_fn = "{}/{}/{}_behav.tsv".format(behav_dir, subject, subject)
        tasks_df = pd.read_csv(tasks_fn, sep="\t")
        
        if subject == 'group':
            af_stair_eb, af_perf_eb = af_df.stair_sem, af_df.perf_sem
            ab_stair_eb, ab_perf_eb = ab_df.stair_sem, ab_df.perf_sem
            eb_end = 'sem'
            
        else:
            af_stair_eb, af_perf_eb = af_df.stair_std, af_df.perf_std
            ab_stair_eb, ab_perf_eb = ab_df.stair_std, ab_df.perf_std
            eb_end = 'std'
            
        
            
        # legend
        if task_num == 0: legend_val = True
        else:legend_val = False

        # subject annotation
        fig.add_annotation(x=ab_df.trial.max()*0.95, y=y_range_stair[1]*0.1, text=subject, showarrow=False, 
                            font_size = template_specs['axes_font_size'], xanchor = 'right', yanchor='middle', row=1, col=task_num+1)
        fig.add_annotation(x=ab_df.trial.max()*0.95, y=y_range_perf[1]*0.1, text=subject, showarrow=False, 
                            font_size = template_specs['axes_font_size'], xanchor = 'right', yanchor='middle', row=2, col=task_num+1)

        
        # staircase attend-bar
        fig.append_trace(go.Scatter(x=ab_df.trial, y=ab_df.stair_mean, name='<i>attend-bar<i>', line_color=ab_line_col, showlegend=legend_val, legendgroup='attend-bar', hovertemplate=hovertemplate_stair),row=1, col=task_num+1)
        fig.append_trace(go.Scatter(x=ab_df.trial, y=ab_df.stair_mean+ab_stair_eb, mode='lines',line_width=0, fillcolor=ab_area_col, showlegend=False, legendgroup='attend-bar', hoverinfo='skip'), row=1, col=task_num+1)
        fig.append_trace(go.Scatter(x=ab_df.trial, y=ab_df.stair_mean-ab_stair_eb, mode='lines',line_width=0, fillcolor=ab_area_col, fill='tonexty', showlegend=False, legendgroup='attend-bar', hoverinfo='skip'),row=1, col=task_num+1)

        # staircase attend-fix
        fig.append_trace(go.Scatter(x=af_df.trial, y=af_df.stair_mean, name='<i>attend-fix<i>', line_color=af_line_col, showlegend=legend_val, legendgroup='attend-fix', hovertemplate=hovertemplate_stair),row=1, col=task_num+1)
        fig.append_trace(go.Scatter(x=af_df.trial, y=af_df.stair_mean+af_stair_eb, mode='lines',line_width=0, fillcolor=af_area_col, showlegend=False, legendgroup='attend-fix', hoverinfo='skip'), row=1, col=task_num+1)
        fig.append_trace(go.Scatter(x=af_df.trial, y=af_df.stair_mean-af_stair_eb, mode='lines',line_width=0, fillcolor=af_area_col, fill='tonexty', showlegend=False, legendgroup='attend-fix', hoverinfo='skip'),row=1, col=task_num+1)

        # perf attend-bar
        fig.append_trace(go.Scatter(x=ab_df.trial, y=ab_df.perf_mean, name='<i>attend-bar<i>', line_color=ab_line_col, showlegend=False, legendgroup='attend-bar', hovertemplate=hovertemplate_perf),row=2, col=task_num+1)
        fig.append_trace(go.Scatter(x=ab_df.trial, y=ab_df.perf_mean+ab_perf_eb, mode='lines',line_width=0, fillcolor=ab_area_col, showlegend=False, legendgroup='Attend-bar', hoverinfo='skip'), row=2, col=task_num+1)
        fig.append_trace(go.Scatter(x=ab_df.trial, y=ab_df.perf_mean-ab_perf_eb, mode='lines',line_width=0, fillcolor=ab_area_col, fill='tonexty', showlegend=False, legendgroup='Attend-bar', hoverinfo='skip'),row=2, col=task_num+1)

        # perf attend-fix
        fig.append_trace(go.Scatter(x=af_df.trial, y=af_df.perf_mean, name='<i>attend-fix<i>', line_color=af_line_col, showlegend=False, legendgroup='attend-fix', hovertemplate=hovertemplate_perf),row=2, col=task_num+1)
        fig.append_trace(go.Scatter(x=af_df.trial, y=af_df.perf_mean+af_perf_eb, mode='lines',line_width=0, fillcolor=af_area_col, showlegend=False, legendgroup='attend-fix', hoverinfo='skip'), row=2, col=task_num+1)
        fig.append_trace(go.Scatter(x=af_df.trial, y=af_df.perf_mean-af_perf_eb, mode='lines',line_width=0, fillcolor=af_area_col, fill='tonexty', showlegend=False, legendgroup='attend-fix', hoverinfo='skip'),row=2, col=task_num+1)


    # across trials
    y_ab_stair_mean = np.asarray([tasks_df['FullScreenAttendBar_stair_mean'],tasks_df['GazeCenterAttendBar_stair_mean'], 
                                  tasks_df['GazeLeftAttendBar_stair_mean'],tasks_df['GazeRightAttendBar_stair_mean']]).reshape(-1)
    y_ab_stair_eb = np.asarray([tasks_df['FullScreenAttendBar_stair_{}'.format(eb_end)], tasks_df['GazeCenterAttendBar_stair_{}'.format(eb_end)], 
                                tasks_df['GazeLeftAttendBar_stair_{}'.format(eb_end)], tasks_df['GazeRightAttendBar_stair_{}'.format(eb_end)]]).reshape(-1)
    y_af_stair_mean = np.asarray([tasks_df['FullScreenAttendFix_stair_mean'],tasks_df['GazeCenterAttendFix_stair_mean'], 
                                  tasks_df['GazeLeftAttendFix_stair_mean'],tasks_df['GazeRightAttendFix_stair_mean']]).reshape(-1)
    y_af_stair_eb = np.asarray([tasks_df['FullScreenAttendFix_stair_{}'.format(eb_end)],tasks_df['GazeCenterAttendFix_stair_{}'.format(eb_end)], 
                                tasks_df['GazeLeftAttendFix_stair_{}'.format(eb_end)],tasks_df['GazeRightAttendFix_stair_{}'.format(eb_end)]]).reshape(-1)
    
    y_ab_perf_mean = np.asarray([tasks_df['FullScreenAttendBar_perf_mean'],tasks_df['GazeCenterAttendBar_perf_mean'], 
                                 tasks_df['GazeLeftAttendBar_perf_mean'],tasks_df['GazeRightAttendBar_perf_mean']]).reshape(-1)
    y_ab_perf_eb = np.asarray([tasks_df['FullScreenAttendBar_perf_{}'.format(eb_end)],tasks_df['GazeCenterAttendBar_perf_{}'.format(eb_end)], 
                               tasks_df['GazeLeftAttendBar_perf_{}'.format(eb_end)],tasks_df['GazeRightAttendBar_perf_{}'.format(eb_end)]]).reshape(-1)
    y_af_perf_mean = np.asarray([tasks_df['FullScreenAttendFix_perf_mean'],tasks_df['GazeCenterAttendFix_perf_mean'], 
                                 tasks_df['GazeLeftAttendFix_perf_mean'],tasks_df['GazeRightAttendFix_perf_mean']]).reshape(-1)
    y_af_perf_eb = np.asarray([tasks_df['FullScreenAttendFix_perf_{}'.format(eb_end)],tasks_df['GazeCenterAttendFix_perf_{}'.format(eb_end)], 
                                 tasks_df['GazeLeftAttendFix_perf_{}'.format(eb_end)],tasks_df['GazeRightAttendFix_perf_{}'.format(eb_end)]]).reshape(-1)
    
    fig.append_trace(go.Bar(x=tasks_label, y=y_ab_stair_mean, showlegend=False, legendgroup='attend-bar', name='<i>attend-bar<i>', marker_color=ab_line_col, hovertemplate=hovertemplate_stair_task,
                            error_y_type='data', error_y_symmetric=False, error_y_array=y_ab_stair_eb, error_y_arrayminus=y_ab_stair_eb),row=1, col=5)

    fig.append_trace(go.Bar(x=tasks_label, y=y_af_stair_mean, showlegend=False, legendgroup='attend-fix', name='<i>attend-fix<i>', marker_color=af_line_col,hovertemplate=hovertemplate_stair_task,
                            error_y_type='data', error_y_symmetric=False, error_y_array=y_af_stair_eb, error_y_arrayminus=y_af_stair_eb),row=1, col=5)
    
    fig.append_trace(go.Bar(x=tasks_label, y=y_ab_perf_mean, showlegend=False, legendgroup='attend-bar', name='<i>attend-bar<i>', marker_color=ab_line_col,hovertemplate=hovertemplate_perf_task,
                            error_y_type='data', error_y_symmetric=False, error_y_array=y_ab_perf_eb, error_y_arrayminus=y_ab_perf_eb),row=2, col=5)
    fig.append_trace(go.Bar(x=tasks_label, y=y_af_perf_mean, showlegend=False, legendgroup='attend-fix', name='<i>attend-fix<i>', marker_color=af_line_col,hovertemplate=hovertemplate_perf_task,
                            error_y_type='data', error_y_symmetric=False, error_y_array=y_af_perf_eb, error_y_arrayminus=y_af_perf_eb),row=2, col=5)
        
    # set axis
    for row in np.arange(rows):
        for col in np.arange(cols):
            fig.update_xaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)
            fig.update_yaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)

    # set figure
    fig.layout.update(xaxis_range =[0,150], xaxis6_range=[0,150], xaxis_title='', xaxis6_title='Trials', xaxis_dtick=75, xaxis6_dtick=75,
                      xaxis2_range=[0,122], xaxis7_range=[0,122], xaxis2_title='', xaxis7_title='Trials', xaxis2_dtick=61, xaxis7_dtick=61,
                      xaxis3_range=[0,122], xaxis8_range=[0,122], xaxis3_title='', xaxis8_title='Trials', xaxis3_dtick=61, xaxis8_dtick=61,
                      xaxis4_range=[0,122], xaxis9_range=[0,122], xaxis4_title='', xaxis9_title='Trials', xaxis4_dtick=61, xaxis9_dtick=61,
                      yaxis_range =y_range_stair, yaxis6_range=y_range_perf, yaxis_title='Kappa (a.u.)',yaxis6_title='Perf (%)',
                      yaxis2_range=y_range_stair, yaxis7_range=y_range_perf, yaxis2_title='',yaxis7_title='',
                      yaxis3_range=y_range_stair, yaxis8_range=y_range_perf, yaxis3_title='',yaxis8_title='',
                      yaxis4_range=y_range_stair, yaxis9_range=y_range_perf, yaxis4_title='',yaxis9_title='',
                      yaxis5_range=y_range_stair, yaxis5_title='',
                      yaxis10_range=y_range_perf, yaxis10_title='',
                      template=fig_template, width=fig_width, height=fig_height, margin_l=50, margin_r=20, margin_t=50, margin_b=70,
                      legend_yanchor='top', legend_y=0.995, legend_xanchor='left', legend_x=0.01, legend_bgcolor='rgba(255,255,255,0)', legend_tracegroupgap=1)
    # show and save figure
    fig.show(config={"displayModeBar": False})
    fig.write_image("{}/{}/{}_behav.pdf".format(behav_dir, subject, subject))
    fig.write_html("{}/{}/{}_behav.html".format(behav_dir, subject, subject),config={"displayModeBar": False})

NameError: name 'attend_task' is not defined